In [3]:
import itertools
import imageio
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import numpy as np
from models import skip

import torch
import torch.optim

#from skimage.measure import compare_psnr
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from utils.denoising_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

imsize =-1
PLOT = True
sigma = 25
sigma_ = sigma/255.

ModuleNotFoundError: No module named 'models.texture_nets'

In [ ]:
def closure():
    
    global i, out_avg, psrn_noisy_last, last_net, net_input
    
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)
    
    out = net(net_input)
    
    # Smoothing
    if out_avg is None:
        out_avg = out.detach()
    else:
        out_avg = out_avg * exp_weight + out.detach() * (1 - exp_weight)
   

    #print(out.shape, img_noisy_torch.shape)
    total_loss = mse(out, img_noisy_torch)
    total_loss.backward()
        
    
    psrn_noisy = compare_psnr(img_noisy_np, out.detach().cpu().numpy()[0]) 
    psrn_gt    = compare_psnr(img_np, out.detach().cpu().numpy()[0]) 
    psrn_gt_sm = compare_psnr(img_np, out_avg.detach().cpu().numpy()[0]) 
    
    # Note that we do not have GT for the "snail" example
    # So 'PSRN_gt', 'PSNR_gt_sm' make no sense
    print ('Iteration %05d    Loss %f   PSNR_noisy: %f   PSRN_gt: %f PSNR_gt_sm: %f' % (i, total_loss.item(), psrn_noisy, psrn_gt, psrn_gt_sm), '\r', end='')
    if  PLOT and i % show_every == 0:
        out_np = torch_to_np(out)
        plot_image_grid([np.clip(out_np, 0, 1), 
                         np.clip(torch_to_np(out_avg), 0, 1)], factor=figsize, nrow=1)
        
        
    
    # Backtracking
    if i % show_every:
        if psrn_noisy - psrn_noisy_last < -5: 
            print('Falling back to previous checkpoint.')

            for new_param, net_param in zip(last_net, net.parameters()):
                net_param.data.copy_(new_param.cuda())

            return total_loss*0
        else:
            last_net = [x.detach().cpu() for x in net.parameters()]
            psrn_noisy_last = psrn_noisy
            
    i += 1

    return total_loss

In [ ]:
# deJPEG 
# fname = 'data/denoising/snail.jpg'
# fname = 'data/denoising/inputA56B9022_periodic.png'

INPUT = 'noise' # 'meshgrid'
pad = 'reflection'
OPT_OVER = 'net' # 'net,input'

reg_noise_std = 1./30. # set to 1./20. for sigma=50
LR = 0.01

OPTIMIZER='adam' # 'LBFGS'
show_every = 100
exp_weight=0.99


## denoising
# noise_type = ['gauss', 'periodic', 'poisson', 'quant']
# numbers = [15,22,23,42,43]
# for item in list(itertools.product(numbers, noise_type)):
for i in range(1):
#     fname = ''.join(['data/denoising/inputA56B90']+[str(item[0])]+['_', item[1] +'.png'])
    fname = "../n2n/data/test/noisy12_Gaussian.png"
    print(fname)
    
    img_noisy_pil = crop_image(get_image(fname, imsize)[0], d=32)
    img_noisy_np = pil_to_np(img_noisy_pil)
    
    # As we don't have ground truth
    img_pil = img_noisy_pil
    img_np = img_noisy_np
    
    if PLOT:
        plot_image_grid([img_np], 4, 5);
        
    
    num_iter = 2400
    input_depth = 3
    figsize = 5 
    
#     net = skip(
#                 input_depth, 3, 
#                 num_channels_down = [8, 16, 32, 64, 128], 
#                 num_channels_up   = [8, 16, 32, 64, 128],
#                 num_channels_skip = [0, 0, 0, 4, 4], 
#                 upsample_mode='bilinear',
#                 need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU')

    net = skip(
                input_depth, 3, 
                channels_down = [8, 16, 32, 64, 128], 
                channels_up   = [8, 16, 32, 64, 128],
                channels_skip = [0, 0, 0, 4, 4], 
                up_mode='bilinear',
                use_sigmoid=True, use_bias=True, padding=pad, activation='LeakyReLU')

    net = net.type(dtype)
    
    net_input = get_noise(input_depth, INPUT, (img_pil.size[1], img_pil.size[0])).type(dtype).detach()
#     print(net_input.shape,img_noisy_np.shape )

    # Compute number of parameters
    s  = sum([np.prod(list(p.size())) for p in net.parameters()]); 
    print ('Number of params: %d' % s)

    # Loss
    mse = torch.nn.MSELoss().type(dtype)

    img_noisy_torch = np_to_torch(img_noisy_np).type(dtype)
    
    net_input_saved = net_input.detach().clone()
    noise = net_input.detach().clone()
    out_avg = None
    last_net = None
    psrn_noisy_last = 0

    i = 0
    
    p = get_params(OPT_OVER, net, net_input)
    optimize(OPTIMIZER, p, closure, LR, num_iter)
    
    
    out_np = torch_to_np(net(net_input))
#     print(net(net_input).shape)
    out_np = np.clip(out_np, 0, 1)
    out_np = out_np * 255
    
    out_np = np.transpose(out_np, (1,2,0)).astype('uint8')
    
#     print(out_np.shape)
    
#     result_dict ='data/denoised_result'
#     os.makedirs(result_dict, exist_ok=True)
#     file_path = os.path.join(result_dict, str(item[0])+item[1]+'.png')
#     imageio.imwrite(file_path, out_np)
    
#     png_im.save('data/denoised_result'+str(item[0])+item[1]+'.png')
